In [51]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
training_generator = ImageDataGenerator(
    rescale = 1/255, # Normalizes pixel values from 0–255 to 0–1.
    shear_range = 0.2, #Skews the image to simulate different angles.
    zoom_range = 0.2,
    horizontal_flip = True
)


In [74]:
print(training_set.class_indices)

{'ripe_lemons': 0, 'unripe_lemons': 1}


In [ ]:
# Load training images from directory
training_set = training_generator.flow_from_directory('training_data',
target_size = (64,64), #Resizes images for consistency.
batch_size = 32,
class_mode = 'binary' #Treats labels as 0 or 1 for two classes.
)

Found 612 images belonging to 2 classes.


In [ ]:
# Normalizes test images (no augmentation—just clean input).
test_generator = ImageDataGenerator (rescale = 1./255)

test_set = test_generator.flow_from_directory('test_data',
target_size=(64,64),
batch_size = 32,
class_mode = 'binary')

Found 150 images belonging to 2 classes.


In [55]:
cnn = tf.keras.models.Sequential()

In [ ]:
# 1st Convolutional layer: Detects patterns.
cnn.add(tf.keras.layers.Conv2D(
    filters=32, # Number of feature detectors.
    kernel_size=3, #3×3 filter.
    activation='relu', #Adds non-linearity.
    input_shape=[64, 64, 3]) #For the first layer only (64×64 RGB).
)

In [ ]:
# Reduces spatial dimensions (64x64 → 32x32), retaining key features.
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

ValueError: Input 0 of layer "max_pooling2d_6" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1)

In [58]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

In [59]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Converts 2D feature maps into a 1D vector for the classifier.
cnn.add(tf.keras.layers.Flatten())

In [ ]:
# Prevents overfitting by randomly “dropping out” 40% of neurons during training.
cnn.add(tf.keras.layers.Dropout(0.4))

In [ ]:
# Fully connected layer with 128 neurons for deep learning.
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
# Output layer: 1 neuron gives a probability between 0 and 1.
# sigmoid is perfect for binary classification.
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer='adam', # Efficient gradient descent. 
loss='binary_crossentropy', # For measuring binary classification error.
metrics=['accuracy'] #Evaluates how many predictions were correct.
)

In [65]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - accuracy: 0.5140 - loss: 0.9045 - val_accuracy: 0.5600 - val_loss: 0.6844
Epoch 2/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - accuracy: 0.5540 - loss: 0.6783 - val_accuracy: 0.4733 - val_loss: 0.6975
Epoch 3/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.5519 - loss: 0.6755 - val_accuracy: 0.6467 - val_loss: 0.6448
Epoch 4/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 126ms/step - accuracy: 0.6897 - loss: 0.5877 - val_accuracy: 0.6667 - val_loss: 0.6170
Epoch 5/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 134ms/step - accuracy: 0.7404 - loss: 0.5253 - val_accuracy: 0.6400 - val_loss: 0.6933
Epoch 6/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.7680 - loss: 0.5208 - val_accuracy: 0.7067 - val_loss: 0.6211
Epoch 7/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 0.8077 - loss: 0.4791 - val_accuracy: 0.6733 - val_loss: 0.6476
Epoch 8/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 0.7811 - loss: 0.5067 - val_accuracy: 0.

In [ ]:
def predict_image(img_path): #Defines a function to load and predict a single lemon image.
    img = image.load_img(img_path, target_size=(64, 64)) #Loads and resizes image.
    img_array = image.img_to_array(img) #Converts image to numerical array.
    img_array = img_array / 255.0 #Normalize pixel values just like training data.
    img_array = np.expand_dims(img_array, axis=0) #Adds an extra dimension (batch size = 1) for prediction.
    prediction = cnn.predict(img_array)[0][0] #Gets the model’s prediction score between 0 and 1.
    print("Confidence: ", prediction) #Shows the raw confidence output from the model.
    return 'Unripened' if prediction >= 0.7 else 'Ripened' #Custom threshold: 0.7+ = Unripened lemon, else Ripened.

In [82]:
print(predict_image('predictions/image6.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Confidence:  0.17373703
Ripened
